In [16]:
text = 'input/pronunciation_01.mp3'
file = text.split('/')[1].split('.')[0]
print(file)

pronunciation_01


In [20]:
import os

list = os.listdir('input')
print(list)

['Pronunciation_01.mp3', 'sentence1.mp3', 'sentence1_4s.mp3', 'sentence1_tts.mp3']


In [44]:
import os
import librosa
import numpy as np
import soundfile as sf

from pydub import AudioSegment
from pydub.silence import split_on_silence

import os
import subprocess
import speech_recognition as sr

import time

# 경로 확인해서 input, chunk_output, final_output 폴더 없으면 생성
if not os.path.exists('input'):
    os.mkdir('input')
    
if not os.path.exists('chunk_output'):
    os.mkdir('chunk_output')

if not os.path.exists('final_output'):
             os.mkdir('final_output')

# 원래 mp3 파일
origin_folder = "input"
origin_file = "Pronunciation_01.mp3"
final_folder = "final_output"

# 무음 0.5초 추가
silence = AudioSegment.silent(duration=500)

def save_file(origin_folder):

    mp3_list = os.listdir(origin_folder)

    for i, origin_mp3 in enumerate(mp3_list):

        origin_file_path = f'{origin_folder}/{origin_mp3}'
        prefix = origin_mp3.split('.')[0]
        
        # 파일 불러오기
        sound = AudioSegment.from_file(os.path.join(os.getcwd(), origin_file_path))
        sound = sound.set_frame_rate(44100).set_channels(1).set_sample_width(2)
    
        # 무음 간격으로 끊기
        audio_chunks = split_on_silence(sound,
            min_silence_len=1000, # 최소 무음 길이 (밀리초 단위)
            silence_thresh=-35, # 무음으로 간주되는 dBFS 값
            keep_silence=500 # 분리된 오디오 조각들 간의 추가적인 무음 길이 (밀리초 단위)
        )
        
        for i, chunk in enumerate(audio_chunks):
            if len(chunk) > 15000:
                chunk = chunk[:15000]
            if i < 3:
                pass
    
            # 강의 안내 방송 이후 시작
            if i >= 3:        
    
                # chunk 담을 폴더
                chunk_folder = "chunk_output"
                
                # wav로 변환된 chunk (speech to text 모듈은 wav만 지원)
                output_file = os.path.join(chunk_folder, f'{prefix}_chunk_{i}.wav')
                chunk.export(output_file, format='wav')
                
                if i >= 3 and i % 2 == 0:
                    try:

                        # 폴더 만들기
                        os.makedirs(f'{final_folder}/{prefix}', exist_ok = True)
                        
                        file1 = AudioSegment.from_wav(f'{chunk_folder}/{prefix}_chunk_{i - 1}.wav')
                        file2 = AudioSegment.from_wav(f'{chunk_folder}/{prefix}_chunk_{i}.wav')
            
                        title = speech_to_text(f'{chunk_folder}/{prefix}_chunk_{i - 1}.wav')
    
                        # 중간 무음 추가
                        merge_file = file1 + silence + file2
    
                        # 현재까지 최종 저장된 파일 개수 확인
                        file_list = os.listdir(f'{final_folder}/{prefix}')
                        file_count = len(file_list)
    
                        # 다시 mp3 로 변환해서 저장
                        merge_file_path = f'{final_folder}/{prefix}/{prefix}_{file_count + 1}_{title}.mp3'
                        merge_file.export(merge_file_path, format="mp3")
                        print(i, merge_file_path)
                    except Exception as e:
                        print(f"Error merging files: {e}")
            # if i >= 5:
            #     break

def speech_to_text(audio_file):
    recognizer = sr.Recognizer()

    with sr.AudioFile(audio_file) as source:
        audio_data = recognizer.record(source)

    try:
        text = recognizer.recognize_google(audio_data, language='en')
        return text
    except sr.UnknownValueError:
        print("음성을 인식할 수 없습니다.")
    except sr.RequestError:
        print("Google Web API에 접근할 수 없습니다.")
    
    return None

def mp3_to_wav(src_file, dest_file):
    sound = AudioSegment.from_mp3(src_file)
    sound.export(dest_file, format="wav")
    pass

def wav_to_mp3(src_file, dest_file):  
    sound = AudioSegment.from_wav(src_file)  
    sound.export(dest_file, format="mp3")  
    pass

if __name__ == '__main__':
    start = time.time()
    save_file(origin_folder)
    print(f"총 소요시간: {time.time() - start: .4f} sec")
    print("완료")
    pass

4 final_output/Pronunciation_01/Pronunciation_01_1_inanimate.mp3
6 final_output/Pronunciation_01/Pronunciation_01_2_animation.mp3
8 final_output/Pronunciation_01/Pronunciation_01_3_animism.mp3
10 final_output/Pronunciation_01/Pronunciation_01_4_inapplicable.mp3
12 final_output/Pronunciation_01/Pronunciation_01_5_incessant.mp3
14 final_output/Pronunciation_01/Pronunciation_01_6_indivisible.mp3
16 final_output/Pronunciation_01/Pronunciation_01_7_inhumane.mp3
18 final_output/Pronunciation_01/Pronunciation_01_8_insatiable.mp3
20 final_output/Pronunciation_01/Pronunciation_01_9_instability.mp3
22 final_output/Pronunciation_01/Pronunciation_01_10_instable.mp3
24 final_output/Pronunciation_01/Pronunciation_01_11_intangible.mp3
26 final_output/Pronunciation_01/Pronunciation_01_12_invariable.mp3
28 final_output/Pronunciation_01/Pronunciation_01_13_a memorial.mp3
30 final_output/Pronunciation_01/Pronunciation_01_14_Memorial.mp3
32 final_output/Pronunciation_01/Pronunciation_01_15_immortal.mp3
34

In [45]:
# chuck 폴더 내 파일 삭제
import os

filePath = 'chunk_output'

for file in os.scandir(filePath):
    os.remove(file.path)

In [42]:
# final_output 폴더 내 파일 삭제 *final_output 하위 폴더의 파일까지 삭제
import os
import shutil

filePath = 'final_output'

for file in os.scandir(filePath):
    shutil.rmtree(file.path)

In [50]:
import os

os.mkdir('input/hi')

In [6]:
AudioSegment.ffmpeg = os.getcwd()+"\\ffmpeg\\bin\\ffmpeg.exe"
print(AudioSegment.ffmpeg)

my_file = Path("Pronunciation_01.mp3")
print(my_file)

C:\Users\황민주\dev\pylibrosa\ffmpeg\bin\ffmpeg.exe
Pronunciation_01.mp3
